# Response to Reviewers: Success Bias

In [1]:
import numpy as np
import os
import sys
sys.path.append('C:\Repos\\temmaretal-20xx')
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
from scipy import stats
import seaborn as sns
import pdb
import config
import utils.online_metrics
from utils.ztools import ZStructTranslator
from utils import offline_metrics
from utils import online_metrics

In [2]:
mk_name = 'Joker'
dates = ['2020-09-12',
            '2020-09-19',
            '2021-07-31'] #add another opt day
runs = [[3, 10, 11, 12, 13],
        [4, 10, 11, 12, 13],
        [3, 7, 8, 9, 10]]
decoderlabels = [['HC', 'RN', 'RK', 'RN', 'RK'],
                    ['HC', 'RK', 'RN', 'RK', 'RN'],
                    ['HC', 'RN', 'RK', 'RN', 'RK']]

In [14]:
trimlength = 5
success_all = {'HC':[],
             'RK':[],
             'RN':[]}
succ_rates = {'HC':[],
             'RK':[],
             'RN':[]}
trial_all = {'HC':[],
             'RK':[],
             'RN':[]}

for d, date in enumerate(dates):
    successes = {'HC':[],
                 'RK':[],
                 'RN':[]}
    trials = {'HC':[],
            'RK':[],
            'RN':[]}    
    for i in np.arange(len(runs[d])):
        run = 'Run-{}'.format(str(runs[d][i]).zfill(3))
        fpath = os.path.join(config.raw_data_dir, mk_name, date, run)

        z = ZStructTranslator(fpath, os.path.join(config.data_dir, 'fits_online'), numChans=96, verbose=True)
        z = z.asdataframe()
        if decoderlabels[d][i] != 'HC': # if not a hand control run, filter by only decoder on trials.
            z = z[z['ClosedLoop'] == True] #make sure decode is on as well
        z = z[trimlength:]

        successes[decoderlabels[d][i]].append(np.sum(z['TrialSuccess']))
        trials[decoderlabels[d][i]].append(len(z['TrialSuccess']))

    for key in succ_rates.keys():
        success_all[key].append(np.sum(successes[key]))
        trial_all[key].append(np.sum(trials[key]))
        succ_rates[key].append(np.sum(successes[key])/np.sum(trials[key]))

Loading previous .npy Z struct . . . 
Finished loading .npy file
Loading previous .npy Z struct . . . 
Finished loading .npy file
Loading previous .npy Z struct . . . 
Finished loading .npy file
Loading previous .npy Z struct . . . 
Finished loading .npy file
Loading previous .npy Z struct . . . 
Finished loading .npy file
Loading previous .npy Z struct . . . 
Finished loading .npy file
Loading previous .npy Z struct . . . 
Finished loading .npy file
Loading previous .npy Z struct . . . 
Finished loading .npy file
Loading previous .npy Z struct . . . 
Finished loading .npy file
Loading previous .npy Z struct . . . 
Finished loading .npy file
Loading previous .npy Z struct . . . 
Finished loading .npy file
Loading previous .npy Z struct . . . 
Finished loading .npy file
Loading previous .npy Z struct . . . 
Finished loading .npy file
Loading previous .npy Z struct . . . 
Finished loading .npy file
Loading previous .npy Z struct . . . 
Finished loading .npy file


In [15]:
succ_rates

{'HC': [0.9974811083123426, 0.9974683544303797, 0.9974874371859297],
 'RK': [0.8741418764302059, 0.8878281622911695, 0.9922680412371134],
 'RN': [0.9955849889624724, 0.9959266802443992, 0.9948717948717949]}

In [24]:
print(np.mean(succ_rates['RN']))
print(np.mean(succ_rates['RK']))
print(trial_all)
print(success_all)


0.9954611546928889
0.9180793599861629
{'HC': [397, 395, 398], 'RK': [437, 419, 388], 'RN': [453, 491, 390]}
{'HC': [396, 394, 397], 'RK': [382, 372, 385], 'RN': [451, 489, 388]}


In [19]:
for i in range(3):
    pdiff = succ_rates['RN'][i] - succ_rates['RK'][i]
    phat = (success_all['RN'][i] + success_all['RK'][i]) / (trial_all['RN'][i] + trial_all['RK'][i])
    sediff = np.sqrt(phat * (1 - phat) * (1 / trial_all['RN'][i] + 1 / trial_all['RK'][i]))

    zscore = pdiff / sediff
    pval = 1 - stats.norm.cdf(np.abs(zscore))
    print(pval)

6.927791673660977e-14
2.9898306053155466e-13
0.3247611462833897


In [18]:
successes['RN']

[244, 144]